In [ ]:
import os
import time

import gqr
import nest_asyncio
import pandas as pd
from dotenv import load_dotenv
from langchain_litellm import ChatLiteLLM
from nemoguardrails import LLMRails, RailsConfig
from tqdm import tqdm

load_dotenv()
nest_asyncio.apply()

In [ ]:
API_KEY = os.getenv("API_KEY")
API_BASE = os.getenv("API_BASE")

domain = gqr.load_id_test_dataset()
ood = gqr.load_ood_test_dataset()

In [ ]:
def batch_is_domain_questions(rails : LLMRails, content: list[str]) -> list:
    results = []
    for i in range(len(content)):
        try:
            _ = rails.generate(messages=[{"role": "user", "content": content[i]}])
            info = rails.explain()
            if str(info.llm_calls[0].completion).lower() == 'yes':
                results.append(False)
            else:
                results.append(True)
        except Exception:
            results.append(False)
    return results

def is_domain_questions(rails : LLMRails, content : str) -> bool:
    try:
        _ = rails.generate(messages=[{"role": "user", "content": content}])
        info = rails.explain()
        return str(info.llm_calls[0].completion).lower() != "yes"
    except Exception:
        return False

In [ ]:
for model_name in ["openai/gpt-4o-mini", "llama3.2:3b", "llama3.1:8b"]:
    config = RailsConfig.from_path("nemo_config")
    llm = ChatLiteLLM(model=model_name, api_key=API_KEY, api_base=API_BASE, max_tokens=5, temperature=0, cache=False)
    rails = LLMRails(config, llm=llm)
    save_path = model_name.replace("/", "_")

    ood["pred"] = ood["text"].apply(is_domain_questions)
    ood.to_csv(f"data/results/nemo_{save_path}_ood_results.csv", index=False)

    domain["pred"] = domain["text"].apply(is_domain_questions)
    domain.to_csv(f"data/results/nemo_{save_path}_domain_results.csv", index=False)

    data = pd.read_csv("data/batch_data.csv")
    batch_results =[]
    data = data['prompt'].tolist()
    for batch_size in tqdm([1, 32, 64, 128, 256]):
        batches = [
            data[i : i + batch_size]
            for i in range(0, len(data), batch_size)
        ]
        for batch in tqdm(batches):
            start_time = time.perf_counter()
            results = batch_is_domain_questions(batch)
            end_time = time.perf_counter()
            elapsed_time = end_time - start_time
            batch_results.append({
                "batch_size": batch_size,
                "time_taken": elapsed_time,
                "results": results,
                "model_name": "nvidia-nemo-gpt4o-mini",
            })
        pd.DataFrame(batch_results).to_csv(f"data/results/batch/nemo_{save_path}_batch.csv", index=False)
